In [40]:
import numpy as np
import scipy.stats as st
from statsmodels.stats.proportion import proportion_confint

Давайте уточним правило трёх сигм. Утверждение: 99.7% вероятностной массы случайной величины $X∼N(\mu,\sigma^2)$ лежит в интервале $\mu\pm c \cdot \sigma $. Чему равно точное значение константы $c$? Округлите ответ до четырёх знаков после десятичной точки. 

In [20]:
st.norm.ppf(1-(1-0.997)/2.)

2.9677379253417717

In [19]:
print("{:.4f}".format(st.norm.ppf(1-(1-0.997)/2.)))

2.9677


В пятилетнем рандомизированном исследовании Гарвардской медицинской школы 11037 испытуемых через день принимали аспирин, а ещё 11034 — плацебо. Исследование было слепым, то есть, испытуемые не знали, что именно они принимают.

За 5 лет инфаркт случился у 104 испытуемых, принимавших аспирин, и у 189 принимавших плацебо.

Оцените, насколько вероятность инфаркта снижается при приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [26]:
n_aspirin = 11037
n_placebo = 11034
i_aspirin = 104
i_placebo = 189

In [30]:
round(i_placebo/n_placebo - i_aspirin/n_aspirin, 4)

0.0077

Постройте теперь 95% доверительный интервал для снижения вероятности инфаркта при приёме аспирина. Чему равна его верхняя граница? Округлите ответ до четырёх знаков после десятичной точки. 

In [41]:
aspirin = np.concatenate((np.ones(i_aspirin), np.zeros(n_aspirin - i_aspirin)))
placebo = np.concatenate((np.ones(i_placebo), np.zeros(n_placebo - i_placebo)))

In [42]:
def proportions_confint_diff_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)   
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [67]:
print("confidence interval: [%.4f, %.4f]" % proportions_confint_diff_ind(placebo, aspirin))

confidence interval: [0.0047, 0.0107]


Продолжим анализировать данные эксперимента Гарвардской медицинской школы.

Для бернуллиевских случайных величин $X∼Ber(p)$ часто вычисляют величину $\frac{p}{1-p}$, которая называется шансами (odds). Чтобы оценить шансы по выборке, вместо $p$ нужно подставить $\hat{p}$. Например, шансы инфаркта в контрольной группе, принимавшей плацебо, можно оценить как

$\frac{\frac{189}{11034}}{1-\frac{189}{11034}} = \frac{189}{11034-189}\approx 0.01741$

Оцените, во сколько раз понижаются шансы инфаркта при регулярном приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки. 

In [47]:
(189/(11034-189)), (104/(11037-104)), round((189/(11034-189)) / (104/(11037-104)), 4)

(0.017427385892116183, 0.009512485136741973, 1.8321)

Величина, которую вы оценили в предыдущем вопросе, называется отношением шансов. Постройте для отношения шансов 95% доверительный интервал с помощью бутстрепа. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

Чтобы получить в точности такой же доверительный интервал, как у нас:

* составьте векторы исходов в контрольной и тестовой выборках так, чтобы в начале шли все единицы, а потом все нули;
* установите random seed=0;
* сделайте по 1000 псевдовыборок из каждой группы пациентов с помощью функции `get_bootstrap_samples`.

In [54]:
def get_odds(sample):
    p = sample.sum()/sample.shape[0]
    odds = p/(1-p)
    return odds

In [50]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [59]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [64]:
np.random.seed(0)

aspirin_odds = np.array(list(map(get_odds, get_bootstrap_samples(aspirin, 1000))))
placebo_odds = np.array(list(map(get_odds, get_bootstrap_samples(placebo, 1000))))

odds_ratio = placebo_odds/aspirin_odds

print("95% confidence interval for the odds ratio:",  stat_intervals(odds_ratio, 0.05))
print("lower bound for the 95% confidence interval of odds ratio: {:.4f}".format(stat_intervals(odds_ratio, 0.05)[0]))

95% confidence interval for the odds ratio: [ 1.44419465  2.34321168]
lower bound for the 95% confidence interval of odds ratio: 1.4442
